In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [7]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("/Users/ishaponugoti/Desktop/DL AD/ADDetection/general/diagnosis_full.csv").drop("Unnamed: 0", axis=1)

In [8]:
diag.head()

,Subject,GroupN,DIAGNOSIS,Phase
0,011_S_0002,1.0,1.0,ADNI1
1,011_S_0002,1.0,1.0,ADNIGO
2,011_S_0002,1.0,1.0,ADNI2
3,011_S_0002,1.0,2.0,ADNI2
4,011_S_0002,1.0,2.0,ADNI3


Below we are combining several clinical datasets.

In [9]:
demo = pd.read_csv("PTDEMOG.csv").rename(columns={"PHASE":"Phase"})
demo.head()

,Phase,PTID,RID,VISCODE,VISCODE2,VISDATE,PTSOURCE,PTGENDER,PTDOB,PTDOBYY,...,PTBIRPR,PTBIRGR,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,NaN,NaN,18,107,2005-08-17,NaN,NaN,NaN,NaN,2005-08-17 00:00:00.0
1,ADNI1,022_S_0001,1,f,f,2005-08-18,1.0,2.0,12/1944,1944.0,...,NaN,NaN,20,10,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
2,ADNI1,011_S_0003,3,sc,sc,2005-08-18,1.0,1.0,05/1924,1924.0,...,NaN,NaN,22,107,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
3,ADNI1,022_S_0004,4,sc,sc,2005-08-18,1.0,1.0,01/1938,1938.0,...,NaN,NaN,24,10,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
4,ADNI1,011_S_0005,5,sc,sc,2005-08-23,1.0,1.0,12/1931,1931.0,...,NaN,NaN,26,107,2005-08-23,NaN,NaN,NaN,NaN,2005-08-23 00:00:00.0


In [10]:
neuro = pd.read_csv("NEUROEXM.csv").rename(columns={"PHASE":"Phase"})
neuro.head()

,Phase,PTID,RID,VISCODE,VISCODE2,VISDATE,NXVISUAL,NXAUDITO,NXTREMOR,NXCONSCI,...,NXOTHER,NXABNORM,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,sc,sc,2005-08-17,1.0,1.0,1.0,1.0,...,1.0,1.0,6,107,2005-08-17,NaN,NaN,NaN,NaN,2005-08-17 00:00:00.0
1,ADNI1,011_S_0003,3,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,...,1.0,1.0,8,107,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
2,ADNI1,022_S_0004,4,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,...,1.0,1.0,10,10,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
3,ADNI1,011_S_0005,5,sc,sc,2005-08-23,1.0,1.0,1.0,1.0,...,1.0,1.0,12,107,2005-08-23,NaN,NaN,NaN,NaN,2005-08-23 00:00:00.0
4,ADNI1,022_S_0007,7,sc,sc,2005-08-25,1.0,1.0,2.0,1.0,...,2.0,1.0,14,10,2005-08-25,NaN,NaN,NaN,NaN,2005-08-25 00:00:00.0


In [11]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [12]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [13]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,PHC_Ethnicity,...,PHC_MEM_PreciseFilter,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,...,1,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,...,1,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,...,1,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,...,1,0.066,0.313,1.0,0.620,0.342,1.0,0.264,0.547,1.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,...,1,0.002,0.336,1.0,0.460,0.374,1.0,0.264,0.547,1.0


In [14]:
# print(neuro.columns)
# print(demo.columns)
# print(clinical.columns)
# print(comb.columns)
# print(diag.columns)
# merged = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()


In [15]:
# diag["Subject"].value_counts()

In [16]:
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv").rename(columns={"PHASE":"Phase"})[["RID", "PTID" , "Phase"]]

In [17]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"])

In [18]:
m = m.rename(columns={"PTID_x":"PTID"})
m = m.drop(["PTID_y"], axis=1)


In [19]:
m = m.merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [20]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [21]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [22]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [23]:
m["PTID"].value_counts()

PTID
067_S_4767    36
141_S_0697    30
131_S_0384    29
067_S_4072    28
126_S_1077    28
              ..
005_S_6093     1
002_S_6070     1
012_S_4247     1
130_S_6035     1
016_S_6949     1
Name: count, Length: 2925, dtype: int64

In [24]:
t = m
t.head()

,RID,PTID,Phase,VISCODE,VISCODE2,VISDATE,PTSOURCE,PTGENDER,PTDOB,PTDOBYY,...,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,GroupN,DIAGNOSIS
0,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,1.0,1.0
1,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,1.0,1.0
2,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0,1.0,1.0
3,3,011_S_0003,ADNI1,sc,sc,2005-08-18,1.0,1.0,05/1924,1924.0,...,0.298,1.0,-0.516,0.238,1.0,-0.237,0.476,1.0,2.0,3.0
4,3,011_S_0003,ADNI1,sc,sc,2005-08-18,1.0,1.0,05/1924,1924.0,...,0.336,1.0,-0.516,0.249,1.0,-0.305,0.484,1.0,2.0,3.0


In [25]:
t.columns
# t["PH"]

Index(['RID', 'PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY',
       ...
       'PHC_EXF_SE', 'PHC_EXF_PreciseFilter', 'PHC_LAN', 'PHC_LAN_SE',
       'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'GroupN', 'DIAGNOSIS'],
      dtype='object', length=117)

In [26]:
#final cols:
l = ['RID', 'PTID', 'GroupN', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

t = t[l]

In [27]:
# t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
#             "update_stamp",  "PTSOURCE", "PTDOBMM","DX"], axis=1) 

In [28]:
t.columns

Index(['RID', 'PTID', 'GroupN', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL', 'NXAUDITO',
       'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER', 'NXHEEL',
       'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXABNORM', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP'],
      dtype='object')

In [29]:
# t = t.fillna(-4)
# t = t.replace("-4", -4)
# cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
# t.drop(cols_to_delete, axis = 1, inplace = True)

In [30]:
len(t.columns)
print(t["PTWORK"])

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
12374   NaN
12375   NaN
12376   NaN
12377   NaN
12378   NaN
Name: PTWORK, Length: 12379, dtype: float64


In [31]:
# t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
#                                                                                                                    "bookkeeper").str.replace("cpa", "accounting")

In [32]:
t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

/var/folders/12/hm8g859d3ml7f4vx9905_51m0000gn/T/ipykernel_10053/2194738997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)


In [33]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

/var/folders/12/hm8g859d3ml7f4vx9905_51m0000gn/T/ipykernel_10053/2309654543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
/var/folders/12/hm8g859d3ml7f4vx9905_51m0000gn/T/ipykernel_10053/2309654543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
/var/folders/12/hm8g859d3ml7f4vx9905_51m0000gn/T/ipykernel_10053/2309654543.py:3: SettingWithCopyWarning: 
A value

In [34]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

/var/folders/12/hm8g859d3ml7f4vx9905_51m0000gn/T/ipykernel_10053/1206247165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')


In [35]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [36]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [37]:
text = ["PTWORK", "CMMED"]

In [38]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,PHC_MEM,NXSENSOR,NXHEEL,PTDOBYY,NXABNORM,NXCONSCI,NXMOTOR,PHC_EXF,PTRACCAT,NXGAIT,NXFINGER,NXPLANTA,GroupN,PTETHCAT,PHC_VSP,PTTLANG,PTCOGBEG,NXNERVE,NXTENDON,PHC_LAN
0,0.405,1.0,1.0,1931.0,1.0,1.0,1.0,0.254,5,1.0,1.0,1.0,1.0,2.0,0.963,1.0,NaN,1.0,1.0,0.293
1,0.364,1.0,1.0,1931.0,1.0,1.0,1.0,0.278,5,1.0,1.0,1.0,1.0,2.0,-0.333,1.0,NaN,1.0,1.0,0.535
2,0.480,1.0,1.0,1931.0,1.0,1.0,1.0,0.419,5,1.0,1.0,1.0,1.0,2.0,0.963,1.0,NaN,1.0,1.0,0.535
3,-0.955,1.0,1.0,1924.0,1.0,1.0,1.0,-0.669,5,1.0,1.0,1.0,2.0,2.0,-0.237,1.0,NaN,1.0,1.0,-0.516
4,-0.657,1.0,1.0,1924.0,1.0,1.0,1.0,-0.137,5,1.0,1.0,1.0,2.0,2.0,-0.305,1.0,NaN,1.0,1.0,-0.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,0.309,1.0,1.0,1960.0,1.0,1.0,1.0,0.503,5,1.0,1.0,1.0,1.0,2.0,0.547,1.0,2020.0,1.0,1.0,0.602
12375,1.248,1.0,1.0,1953.0,1.0,1.0,1.0,1.230,4,1.0,1.0,1.0,1.0,2.0,0.963,1.0,2020.0,1.0,2.0,0.886
12376,0.390,1.0,1.0,1945.0,1.0,1.0,1.0,0.503,5,1.0,1.0,1.0,1.0,2.0,0.547,1.0,2017.0,1.0,1.0,0.602
12377,0.390,1.0,1.0,1942.0,1.0,1.0,1.0,0.503,6,1.0,1.0,2.0,0.0,1.0,-0.228,1.0,NaN,1.0,1.0,0.071


In [39]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXSENSOR
NXHEEL
NXABNORM
NXCONSCI
NXMOTOR
PTRACCAT
NXGAIT
NXFINGER
NXPLANTA
GroupN
PTETHCAT
PTTLANG
NXNERVE
NXTENDON


In [40]:
# to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]
# t = t.drop(to_del, axis=1)

In [41]:
# quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
# t[quant]

In [42]:
# cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [43]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [44]:
# t[l]

In [45]:
dfs = []

In [46]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [48]:
dfs

[       PTGENDER_-4.0  PTGENDER_1.0  PTGENDER_2.0
 0              False          True         False
 1              False          True         False
 2              False          True         False
 3              False          True         False
 4              False          True         False
 ...              ...           ...           ...
 12374          False          True         False
 12375          False         False          True
 12376          False          True         False
 12377          False         False          True
 12378          False         False          True
 
 [12379 rows x 3 columns],
        PTWORK_-4
 0           True
 1           True
 2           True
 3           True
 4           True
 ...          ...
 12374       True
 12375       True
 12376       True
 12377       True
 12378       True
 
 [12379 rows x 1 columns],
        PTHOME_-4.0  PTHOME_1.0  PTHOME_2.0  PTHOME_3.0  PTHOME_4.0  \
 0            False        True       False       False

In [ ]:
cat = pd.concat(dfs, axis=1)

In [ ]:
# t[quant]

In [ ]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,...,NXSENSOR_1.0,NXSENSOR_2.0,NXFINGER_1.0,NXFINGER_2.0,NXPLANTA_1.0,NXPLANTA_2.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0
0,False,True,False,True,False,True,False,False,False,False,...,True,False,True,False,True,False,False,False,True,False
1,False,True,False,True,False,True,False,False,False,False,...,True,False,True,False,True,False,False,False,True,False
2,False,True,False,True,False,True,False,False,False,False,...,True,False,True,False,True,False,False,False,True,False
3,False,True,False,True,False,False,False,True,False,False,...,True,False,True,False,True,False,False,False,True,False
4,False,True,False,True,False,False,False,True,False,False,...,True,False,True,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,False,True,False,True,False,True,False,False,False,False,...,True,False,True,False,True,False,False,False,True,False
12375,False,False,True,True,False,True,False,False,False,False,...,True,False,True,False,True,False,False,False,True,False
12376,False,True,False,True,False,False,False,True,False,False,...,True,False,True,False,True,False,False,False,True,False
12377,False,False,True,True,False,True,False,False,False,False,...,True,False,True,False,False,True,False,True,False,False


In [ ]:
t[["PTID","RID", "Phase", "GroupN"]]

,PTID,RID,Phase,GroupN
0,011_S_0002,2,ADNI1,1.0
1,011_S_0002,2,ADNI1,1.0
2,011_S_0002,2,ADNI1,1.0
3,011_S_0003,3,ADNI1,2.0
4,011_S_0003,3,ADNI1,2.0
...,...,...,...,...
12374,035_S_7121,7121,ADNI3,1.0
12375,082_S_7117,7117,ADNI3,1.0
12376,013_S_7110,7110,ADNI3,1.0
12377,035_S_7123,7123,ADNI3,0.0


In [ ]:
c = pd.concat([t[["PTID", "RID", "Phase", "GroupN"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [ ]:
c

,PTID,RID,Phase,GroupN,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTHOME_-4.0,PTHOME_1.0,...,PTETHCAT_2.0,PTETHCAT_3.0,PTDOBYY,PHC_MEM,PHC_EXF,PTRACCAT,PTETHCAT,PTCOGBEG,PHC_VSP,PHC_LAN
0,011_S_0002,2,ADNI1,1.0,False,True,False,True,False,True,...,True,False,1931.0,0.405,0.254,5,2.0,NaN,0.963,0.293
1,011_S_0002,2,ADNI1,1.0,False,True,False,True,False,True,...,True,False,1931.0,0.364,0.278,5,2.0,NaN,-0.333,0.535
2,011_S_0002,2,ADNI1,1.0,False,True,False,True,False,True,...,True,False,1931.0,0.480,0.419,5,2.0,NaN,0.963,0.535
3,011_S_0003,3,ADNI1,2.0,False,True,False,True,False,False,...,True,False,1924.0,-0.955,-0.669,5,2.0,NaN,-0.237,-0.516
4,011_S_0003,3,ADNI1,2.0,False,True,False,True,False,False,...,True,False,1924.0,-0.657,-0.137,5,2.0,NaN,-0.305,-0.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,035_S_7121,7121,ADNI3,1.0,False,True,False,True,False,True,...,True,False,1960.0,0.309,0.503,5,2.0,2020.0,0.547,0.602
12375,082_S_7117,7117,ADNI3,1.0,False,False,True,True,False,True,...,True,False,1953.0,1.248,1.230,4,2.0,2020.0,0.963,0.886
12376,013_S_7110,7110,ADNI3,1.0,False,True,False,True,False,False,...,True,False,1945.0,0.390,0.503,5,2.0,2017.0,0.547,0.602
12377,035_S_7123,7123,ADNI3,0.0,False,False,True,True,False,True,...,False,False,1942.0,0.390,0.503,6,1.0,NaN,-0.228,0.071


In [ ]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["GroupN"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [ ]:
c.to_csv("clinical.csv")

In [ ]:
# #reading in the overlap test set
# ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

# #removing ids from the overlap test set
# c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "GroupN", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["GroupN"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
print(len(l))


34


In [ ]:
pd.DataFrame(X_train).to_pickle("X_train_c.pkl")
pd.DataFrame(y_train).to_pickle("y_train_c.pkl")

pd.DataFrame(X_test).to_pickle("X_test_c.pkl")
pd.DataFrame(y_test).to_pickle("y_test_c.pkl")